In [1]:

from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
mydb = client.cancer
my_collection = mydb.cancer

In [3]:
client.list_database_names()

['TestA', 'TestB', 'Testc', 'admin', 'config', 'local']

In [5]:
mydb=client.Testc
mydb.list_collection_names()

['data _can']

In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import json 
import csv 
df = pd.read_csv('Breast_cancer_data.csv')
df.head(10)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
5,12.45,15.70,82.57,477.1,0.12780,0
6,18.25,19.98,119.60,1040.0,0.09463,0
7,13.71,20.83,90.20,577.9,0.11890,0
8,13.00,21.82,87.50,519.8,0.12730,0
9,12.46,24.04,83.97,475.9,0.11860,0


In [3]:
df_json = json.loads(df.to_json(orient='records'))
record_id = mydb.basedata.insert_many(df_json)
print (record_id)
print (mydb.collection_names())

['basedata']


C:\Users\pkdut\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  after removing the cwd from sys.path.


In [4]:
mydb.basedata.find()
mycol = mydb["basedata"]
for x in mycol.find():
  print(x)


{'_id': ObjectId('5eb811c059db4781673cdf32'), 'mean_radius': 17.99, 'mean_texture': 10.38, 'mean_perimeter': 122.8, 'mean_area': 1001.0, 'mean_smoothness': 0.1184, 'diagnosis': 0}
{'_id': ObjectId('5eb811c059db4781673cdf33'), 'mean_radius': 20.57, 'mean_texture': 17.77, 'mean_perimeter': 132.9, 'mean_area': 1326.0, 'mean_smoothness': 0.08474, 'diagnosis': 0}
{'_id': ObjectId('5eb811c059db4781673cdf34'), 'mean_radius': 19.69, 'mean_texture': 21.25, 'mean_perimeter': 130.0, 'mean_area': 1203.0, 'mean_smoothness': 0.1096, 'diagnosis': 0}
{'_id': ObjectId('5eb811c059db4781673cdf35'), 'mean_radius': 11.42, 'mean_texture': 20.38, 'mean_perimeter': 77.58, 'mean_area': 386.1, 'mean_smoothness': 0.1425, 'diagnosis': 0}
{'_id': ObjectId('5eb811c059db4781673cdf36'), 'mean_radius': 20.29, 'mean_texture': 14.34, 'mean_perimeter': 135.1, 'mean_area': 1297.0, 'mean_smoothness': 0.1003, 'diagnosis': 0}
{'_id': ObjectId('5eb811c059db4781673cdf37'), 'mean_radius': 12.45, 'mean_texture': 15.7, 'mean_peri

In [30]:
df.describe

<bound method NDFrame.describe of      mean_radius  mean_texture  mean_perimeter  mean_area  mean_smoothness  \
0          17.99         10.38          122.80     1001.0          0.11840   
1          20.57         17.77          132.90     1326.0          0.08474   
2          19.69         21.25          130.00     1203.0          0.10960   
3          11.42         20.38           77.58      386.1          0.14250   
4          20.29         14.34          135.10     1297.0          0.10030   
..           ...           ...             ...        ...              ...   
564        21.56         22.39          142.00     1479.0          0.11100   
565        20.13         28.25          131.20     1261.0          0.09780   
566        16.60         28.08          108.30      858.1          0.08455   
567        20.60         29.33          140.10     1265.0          0.11780   
568         7.76         24.54           47.92      181.0          0.05263   

     diagnosis  
0           

In [9]:
df.groupby('diagnosis').count()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness
diagnosis,,,,,
0,212,212,212,212,212
1,357,357,357,357,357


In [5]:
feature_cols = ['mean_radius', 'mean_texture', 'mean_perimeter', 'mean_area','mean_smoothness']
X = df[feature_cols] # Features
y = df.diagnosis # Target variable
print("Feature shape:" , X.shape)
print("Target shape:" , y.shape)

Feature shape: (569, 5)
Target shape: (569,)


In [71]:
#split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=109)
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9186046511627907


In [67]:
confusion_matrix(y_test, y_pred)

array([[26,  5],
       [ 2, 53]], dtype=int64)

In [68]:
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='macro')
print('Precision: %.3f' % precision)

Precision: 0.921


In [69]:
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred, average='macro')
print('recall: %.3f' % recall)

recall: 0.901


In [47]:
def prediction(new_feature):
    predict = classifier.predict(new_feature)
    if(predict==0):
       print('Breast Cancer: Negative')
       return "Negative"
    elif(predict==1):
       print('Breast Cancer: Positive')
       return "Positive"

In [45]:
#Collect feature/sample f
record = {
        "patient": "Uzzal Mondal",
        "mean_radius": 17.99,
        "mean_texture":10.38, 
        "mean_perimeter":122.80, 
        "mean_area":1001.0,
        "mean_smoothness":0.11840,
        "date" : datetime.datetime.utcnow()
        }

target_predict = [[ record['mean_radius'],record['mean_texture'],record['mean_perimeter'],record['mean_area'],record['mean_smoothness']]]
record['Breast Cancer'] = prediction(target_predict)
record_id = mydb.prediction.insert(record)
print (record_id)
print (mydb.collection_names())

Breast Cancer: Negative


/home/uzzal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


5eb6a3e529920de46ce53cf5
['prediction', 'mytable', 'basedata']


/home/uzzal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  app.launch_new_instance()


In [46]:
mydb.prediction.find()
mycol = mydb["prediction"]
for x in mycol.find():
  print(x)

{'_id': ObjectId('5eb6a3e529920de46ce53cf5'), 'patient': 'Uzzal Mondal', 'mean_radius': 17.99, 'mean_texture': 10.38, 'mean_perimeter': 122.8, 'mean_area': 1001.0, 'mean_smoothness': 0.1184, 'date': datetime.datetime(2020, 5, 9, 12, 36, 53, 473000), 'Breast Cancer': None}


In [48]:
#Collect feature/sample f
record = {
        "patient": "Jit Dutta",
        "mean_radius": 18.99,
        "mean_texture":10.38, 
        "mean_perimeter":122.80, 
        "mean_area":1001.0,
        "mean_smoothness":0.11840,
        "date" : datetime.datetime.utcnow()
        }

target_predict = [[ record['mean_radius'],record['mean_texture'],record['mean_perimeter'],record['mean_area'],record['mean_smoothness']]]
record['Breast Cancer'] = prediction(target_predict)
record_id = mydb.prediction.insert(record)
print (record_id)
print (mydb.collection_names())

Breast Cancer: Negative
5eb6a4e229920de46ce53cf6
['prediction', 'mytable', 'basedata']


/home/uzzal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  
/home/uzzal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  app.launch_new_instance()


In [49]:
mydb.prediction.find()
mycol = mydb["prediction"]
for x in mycol.find():
  print(x)

{'_id': ObjectId('5eb6a3e529920de46ce53cf5'), 'patient': 'Uzzal Mondal', 'mean_radius': 17.99, 'mean_texture': 10.38, 'mean_perimeter': 122.8, 'mean_area': 1001.0, 'mean_smoothness': 0.1184, 'date': datetime.datetime(2020, 5, 9, 12, 36, 53, 473000), 'Breast Cancer': None}
{'_id': ObjectId('5eb6a4e229920de46ce53cf6'), 'patient': 'Jit Dutta', 'mean_radius': 18.99, 'mean_texture': 10.38, 'mean_perimeter': 122.8, 'mean_area': 1001.0, 'mean_smoothness': 0.1184, 'date': datetime.datetime(2020, 5, 9, 12, 41, 6, 79000), 'Breast Cancer': 'Negative'}
